In [25]:
# Import Libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
import pickle
from sklearn import linear_model


In [26]:
#========================================================== Helper Functions ========================================================================

# Load Model
def load(file_path):
    loaded_model = pickle.load(open(file_path, 'rb'))
    return loaded_model


def remove_value(data, column_name, string_to_replace, replace_value='', change_dataType=True):
  data[column_name] = data[column_name].str.replace(string_to_replace, replace_value)
  if change_dataType:
    data[column_name] = data[column_name].astype(int)


def fill_nulls(data):
     mode_dict = {'brand': 'ASUS', 'processor_brand': 'Intel', 'processor_name': 'Core i5', 'processor_gnrtn': '11th', 'ram_gb': '8 GB', 'ram_type': 'DDR4', 'ssd': '512 GB', 'hdd': '0 GB', 'os': '64-bit Windows', 'graphic_card_gb': '0 GB', 'weight': 'Casual', 'warranty': '1 year', 'Touchscreen': 'No', 'msoffice': 'No', 'Price': 64990, 'rating': 'Good Rating', 'Number of Ratings': 0, 'Number of Reviews': 0}
     for col, mode in mode_dict.items():
        data[col].fillna(mode, inplace=True)

# Handle Missing Values
def remove_outliers(data):

    lower_bound = {'brand': -5.0, 'processor_name': -2.5, 'processor_gnrtn': 8.5, 'ram_gb': -2.0, 'ram_type': 3.0, 'ssd': -128.0, 'hdd': -768.0, 'graphic_card_gb': -3.0, 'weight': -0.5, 'warranty': -1.5, 'Touchscreen': 0.0, 'msoffice': -1.5, 'Price': -19869.0, 'Number of Ratings': -231.0, 'Number of Reviews': -29.25, 'os_bits': 64.0, 'AMD': 0.0, 'Intel': -1.5, 'M1': 0.0, 'DOS': 0.0, 'Mac': 0.0, 'Windows': 1.0}
    upper_bound = {'brand': 11.0, 'processor_name': 9.5, 'processor_gnrtn': 12.5, 'ram_gb': 14.0, 'ram_type': 3.0, 'ssd': 896.0, 'hdd': 1280.0, 'graphic_card_gb': 5.0, 'weight': 3.5, 'warranty': 2.5, 'Touchscreen': 0.0, 'msoffice': 2.5, 'Price': 155675.0, 'Number of Ratings': 385.0, 'Number of Reviews': 48.75, 'os_bits': 64.0, 'AMD': 0.0, 'Intel': 2.5, 'M1': 0.0, 'DOS': 0.0, 'Mac': 0.0, 'Windows': 1.0}

    s = ['ram_type' ,'Touchscreen', 'os_bits' , 'AMD' , 'M1' , 'DOS' , 'Mac' , 'Windows' ]

    # Replace the outliers with lower_bound or upper_bound values
    for col in data.iloc[:,:-1]:
        if col not in s:
            data.loc[data[col] < lower_bound[col], col] = lower_bound[col]
            data.loc[data[col] > upper_bound[col], col] = upper_bound[col]

    return data


# Feature selection using chi-squared
def feature_selection(data):
    selected_indices = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
    data = data.iloc[: , selected_indices]
    return data

# Execute Model
def execute_model(model, X_test, Y_test, model_name):
    Y_test_pred = model.predict(X_test)

    if model_name in ['Linear Regression', 'Polynomial Regression']:
        mse = mean_squared_error(Y_test, Y_test_pred)
        r2 = r2_score(Y_test, Y_test_pred)
        print(f"{model_name}:\nMSE: {mse}\nR2 Score: {r2}\n")
    else:
        print("Only regression models are supported.")

In [27]:
# Feature Encoder
def feature_encoder(X, cols):
    for c in cols:
        lbl = LabelEncoder()
        lbl.fit(list(X[c].values))
        X[c] = lbl.transform(list(X[c].values))
    return X

In [28]:
# Preprocess Data
def preprocess_data(data, encoders, one_hot_encoders):
    # filling nulls
    fill_nulls(data)
    print("passed fill nulls")
    # processor_gnrtn
    data['processor_gnrtn'].replace('Not Available', '11th', inplace = True)
    print("passed replace not availabel")
    remove_value(data, 'processor_gnrtn', 'th')
    print("passed remove th from processor gnrtn")

    # ram_gb
    remove_value(data,'ram_gb' ,' GB')

    # ssd
    remove_value(data,'ssd' ,' GB')

    # hdd
    remove_value(data,'hdd' ,' GB')

    # os
    data[['os_bits', 'os_type']] = data['os'].str.split(' ', expand=True)
    data = data.drop('os', axis = 1)
    remove_value(data,'os_bits' ,'-bit')

    # graphic_card_gb
    remove_value(data,'graphic_card_gb' ,' GB')

    # warranty
    remove_value(data,'warranty' ,'No warranty', '0', False)
    remove_value(data,'warranty' ,' years', '', False)
    remove_value(data,'warranty' ,' year')

    # Touchscreen
    remove_value(data,'Touchscreen' ,'No', '0', False)
    remove_value(data,'Touchscreen' ,'Yes', '1')

    # msoffice
    remove_value(data,'msoffice' ,'No', '0', False)
    remove_value(data,'msoffice' ,'Yes', '1')

    # rating
    remove_value(data,'rating' ,' stars', '', False)
    remove_value(data,'rating' ,' star')



    # ram_type
    ram_performance_mapping = {
        'LPDDR4X': 5,
        'LPDDR4': 4,
        'DDR4': 3,
        'DDR3': 2,
        'DDR5': 6,
        'LPDDR3': 1
    }

    data['ram_type'] = data['ram_type'].map(ram_performance_mapping).fillna(7)
    # data['ram_type'].value_counts()

    # weight
    weight = {
        'Casual':2,
        'ThinNlight':1,
        'Gaming':3,
    }
    data['weight'] = data['weight'].map(weight).fillna(4)


    brand_mapping = {
    'ASUS': 1,
    'Lenovo': 5,
    'HP': 4,
    'APPLE': 0,
    'DELL': 3,
    'acer': 7,
    'MSI': 6,
    'Avita': 2
    }

    data['brand'] = data['brand'].map(brand_mapping).fillna(8)


    processor_name_mapping = {
    'Pentium Quad': 6,
    'Core i5': 2,
    'Celeron Dual': 0,
    'Core i3': 1,
    'Ryzen 7': 9,
    'M1': 5,
    'Ryzen 5': 8,
    'Core i7': 3,
    'Ryzen 3': 7,
    'Ryzen 9': 10,
    'Core i9': 4
    }

    data['processor_name'] = data['processor_name'].map(processor_name_mapping).fillna(11)
    # Apply One-Hot Encoders to data
    one_hot_encoded = pd.get_dummies(data['processor_brand'])
    one_hot_encoded = one_hot_encoded.astype(int)

    if set(['AMD', 'Intel', 'M1']).intersection(set(data['processor_brand'])):
        data = data.drop('processor_brand', axis = 1)
        data = pd.concat([data, one_hot_encoded], axis=1)
    else:
        data = data.drop('processor_brand', axis = 1)


    missing_column = 'AMD'
    if missing_column not in data.columns:
        data[missing_column] = 0

    missing_column = 'Intel'
    if missing_column not in data.columns:
        data[missing_column] = 0

    missing_column = 'M1'
    if missing_column not in data.columns:
        data[missing_column] = 0


    if set(['DOS', 'Mac', 'Windows']).intersection(set(data['os_type'])):
        one_hot_encoded = pd.get_dummies(data['os_type'])
        one_hot_encoded = one_hot_encoded.astype(int)
        data = data.drop('os_type', axis = 1)
        data = pd.concat([data, one_hot_encoded], axis=1)
    else:
        data = data.drop('os_type', axis = 1)

    missing_column = 'DOS'
    if missing_column not in data.columns:
        data[missing_column] = 0

    missing_column = 'Mac'
    if missing_column not in data.columns:
        data[missing_column] = 0

    missing_column = 'Windows'
    if missing_column not in data.columns:
        data[missing_column] = 0

    # Replace target column
    rating_col = data['rating']
    data = data.drop('rating', axis = 1)
    data = pd.concat([data, rating_col], axis=1)

    data = remove_outliers(data)


    return data

In [29]:
# Handle Missing Values
def handle_missing_values(data):
    columns_to_exclude = ['ram_type', 'Touchscreen', 'os_bits', 'AMD', 'M1', 'DOS', 'Mac', 'Windows', 'rating','Price']
    data_filtered = data.drop(columns=columns_to_exclude)


    lower_bound = {'brand': -5.0, 'processor_name': -2.5, 'processor_gnrtn': 8.5, 'ram_gb': -2.0, 'ram_type': 3.0, 'ssd': -128.0, 'hdd': -768.0, 'graphic_card_gb': -3.0, 'weight': -0.5, 'warranty': -1.5, 'Touchscreen': 0.0, 'msoffice': -1.5, 'Price': -19869.0, 'Number of Ratings': -231.0, 'Number of Reviews': -29.25, 'os_bits': 64.0, 'AMD': 0.0, 'Intel': -1.5, 'M1': 0.0, 'DOS': 0.0, 'Mac': 0.0, 'Windows': 1.0}
    upper_bound = {'brand': 11.0, 'processor_name': 9.5, 'processor_gnrtn': 12.5, 'ram_gb': 14.0, 'ram_type': 3.0, 'ssd': 896.0, 'hdd': 1280.0, 'graphic_card_gb': 5.0, 'weight': 3.5, 'warranty': 2.5, 'Touchscreen': 0.0, 'msoffice': 2.5, 'Price': 155675.0, 'Number of Ratings': 385.0, 'Number of Reviews': 48.75, 'os_bits': 64.0, 'AMD': 0.0, 'Intel': 2.5, 'M1': 0.0, 'DOS': 0.0, 'Mac': 0.0, 'Windows': 1.0}

    s = ['ram_type' ,'Touchscreen', 'os_bits' , 'AMD' , 'M1' , 'DOS' , 'Mac' , 'Windows' ]

    # Replace the outliers with lower_bound or upper_bound values
    for col in data.iloc[:,:-1]:
        if col not in s:
            data.loc[data[col] < lower_bound[col], col] = lower_bound[col]
            data.loc[data[col] > upper_bound[col], col] = upper_bound[col]

    # sns.boxplot(data=data,palette='rainbow',orient='h')
    data_filtered = data.drop(columns=columns_to_exclude)
    return data

In [30]:
    # Load Model
    linear_regression_model = load('MS1 test script regression.sav')

    # Load saved LabelEncoders
    encoders = load('ms1 regression encoders.sav')
    one_hot_encoders = load('ms1 regression one hot encoders.sav')

    # Load Test Data
    test_data = pd.read_csv('/content/csv test script.csv')

    # Preprocess Test Data
    test_data = preprocess_data(test_data, encoders, one_hot_encoders)

    # Ensure test data has the same features as training data
    # Ensure test data has the same features as training data
    X_test = test_data[['brand', 'processor_name', 'processor_gnrtn', 'ram_gb', 'ram_type',
       'ssd', 'hdd', 'graphic_card_gb', 'weight', 'warranty', 'Touchscreen',
       'msoffice', 'Price', 'Number of Ratings', 'Number of Reviews',
       'os_bits', 'AMD', 'Intel', 'M1', 'DOS', 'Mac', 'Windows', 'rating']]

    X_test = X_test.iloc[:, :-1]
    # Execute Model
    Y_test = test_data['rating']
    execute_model(linear_regression_model, X_test, Y_test, "Linear Regression")


passed fill nulls
passed replace not availabel
passed remove th from processor gnrtn
Linear Regression:
MSE: 0.012009678236754423
R2 Score: nan



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
